In [143]:
import os
os.getcwd()

#Set new working directory
os.chdir("C:\SunCulture\SunCulture\Python\credit_performance")

In [145]:
# the libraries we need
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import (RFE,SelectKBest,SelectPercentile)
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np

In [146]:
#read the data
data = pd.read_csv('cds_data.csv')
data.head()

,customer_id,customer_data_survey_two_id,customer_data_survey_id,account_id,id_account,willingness_to_record,same_phone_number,land_owner,farming_duration,county_id,...,Netincome2.3,birth_date,gender,duration_with_phone_number,county_of_residence,duration_in_location,relation_to_next_of_kin,willing_to_share_phone_number,house_hold_number,bad_90
0,43e7b188,001f6753,7058f03e,0b38156f,1,1.0,1.0,1.0,0-1 year,NaN,...,0.0,31 to 45 years,Male,More than two years,2AEB5F2F,More than two years,Wife,1.0,4-7,0.0
1,RAF2VMEU,00e2cad6,LI8GLXD6,75B5UP65,2,1.0,1.0,1.0,0-1 year,7F919359,...,0.0,31 to 45 years,Male,More than two years,Nairobi,More than two years,Wife,1.0,1,NaN
2,Q2GDVZO7,00f68935,H67DCDCF,ZWSIULX9,3,1.0,1.0,1.0,0-1 year,E953A7E2,...,0.0,31 to 45 years,Male,More than two years,Nairobi,More than two years,Mother,1.0,2-3,0.0
3,5ab69a67,00fa561d,22c00510,8a65c4c2,4,1.0,1.0,0.0,0-1 year,NaN,...,100.0,31 to 45 years,Female,More than two years,E953A7E2,More than two years,Daughter,1.0,4-7,1.0
4,d8c71ac6,01f7fd52,cc3a7520,3767faff,5,1.0,1.0,1.0,0-1 year,560E094E,...,0.0,18 to 30 years,Male,More than two years,560E094E,More than two years,Sister,1.0,2-3,0.0


In [147]:
#EDA
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8464 entries, 0 to 8463
Data columns (total 80 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   customer_id                                      8464 non-null   object 
 1   customer_data_survey_two_id                      8464 non-null   object 
 2   customer_data_survey_id                          8464 non-null   object 
 3   account_id                                       8460 non-null   object 
 4   id_account                                       8464 non-null   int64  
 5   willingness_to_record                            8341 non-null   float64
 6   same_phone_number                                8462 non-null   float64
 7   land_owner                                       8341 non-null   float64
 8   farming_duration                                 8464 non-null   object 
 9   county_id                     

In [148]:
#select specific columns
df= data[['id_account','willingness_to_record','same_phone_number','land_owner','farming_duration',
'animal_numbers','harvest_cycles_per_year','crops_grown','fruit_vegetables_grown','pests_diseases_control',
'machinery_ownership','farming_acreage','product_purpose','customer_source','family_descision_maker','other_water_source',
'hours_fetching_water','pay_for_water','pay_for_pumping','water_needs_capicity','tank_capacity','electricity_connectivity',
'household_number','financial_dependants_children_adults','working_age_adults','pensioners','occupation_of_head',
'main_income_source','members_during_peak_season','compensation_type','occupation_length','number_of_businesses_before',
'average_monthly_income','duration_with_same_employer','number_of_jobs_before','salary_amount','other_income_source',
'frequent_monthly_expense','remaining_income_after_expenses','number_of_loans','number_of_outstanding_loans',
'total_outstanding_loan_amount','remaining_payment_months','preferred_banking_method','percentage_who_pay_suppliers_on_time',
'percentage_who_default','expenditure_after_winning','overall_risk_assessment_by_relationship_manager','sum_householdmember',
'sum_otherincome','loaninstallment_C','netincome1_business','netincome1_salary','netincome2_business','netincome2_salary',
'netincome3','affordability1','affordability2','affordability3','housemembers_CC','otherloans_CC','netincome1.2',
'netincome1.3','Netincome2.3','birth_date','gender','duration_with_phone_number','county_of_residence',
'duration_in_location','relation_to_next_of_kin','willing_to_share_phone_number','house_hold_number','bad_90']]


In [149]:
#list the numerical columns
df_numerics_only = df.select_dtypes(include=np.number)
df_numerics_only

,id_account,willingness_to_record,same_phone_number,land_owner,harvest_cycles_per_year,hours_fetching_water,pay_for_water,pay_for_pumping,water_needs_capicity,tank_capacity,...,affordability1,affordability2,affordability3,housemembers_CC,otherloans_CC,netincome1.2,netincome1.3,Netincome2.3,willing_to_share_phone_number,bad_90
0,1,1.0,1.0,1.0,2,0.0,0.0,0.0,7000.0,5000.0,...,-34.01,-34.01,-33.86,0,0,100.0,0.0,0.0,1.0,0.0
1,2,1.0,1.0,1.0,0,0.0,0.0,0.0,21000.0,3000.0,...,0.00,0.00,0.00,0,0,0.0,0.0,0.0,1.0,NaN
2,3,1.0,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0,...,-9.21,-9.21,-9.20,0,0,100.0,0.0,0.0,1.0,0.0
3,4,1.0,1.0,0.0,323232,14.0,NaN,7000.0,5000.0,0.0,...,-6.14,-28.61,-28.67,0,0,0.0,0.0,100.0,1.0,1.0
4,5,1.0,1.0,1.0,0,0.0,0.0,0.0,0.0,0.0,...,-25.60,-25.60,-25.55,1,0,100.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8459,8460,1.0,1.0,1.0,2,NaN,NaN,NaN,52500.0,2500.0,...,-215.00,-215.00,-215.00,0,0,0.0,0.0,0.0,1.0,0.0
8460,8461,1.0,1.0,1.0,4,10.0,0.0,0.0,10000.0,5000.0,...,-15.35,-15.35,-15.33,1,0,100.0,0.0,0.0,1.0,0.0
8461,8462,1.0,1.0,1.0,2,5.0,0.0,0.0,3000.0,0.0,...,-46.22,-46.22,-45.99,0,0,100.0,0.0,0.0,0.0,0.0
8462,8463,1.0,1.0,1.0,3,NaN,NaN,NaN,10000.0,1000.0,...,-15.76,-15.76,-15.71,1,0,100.0,0.0,0.0,1.0,1.0


In [150]:
colnames_numerics_only = df.select_dtypes(include=np.number).columns.tolist()
colnames_numerics_only

['id_account',
 'willingness_to_record',
 'same_phone_number',
 'land_owner',
 'harvest_cycles_per_year',
 'hours_fetching_water',
 'pay_for_water',
 'pay_for_pumping',
 'water_needs_capicity',
 'tank_capacity',
 'financial_dependants_children_adults',
 'working_age_adults',
 'pensioners',
 'average_monthly_income',
 'salary_amount',
 'remaining_income_after_expenses',
 'total_outstanding_loan_amount',
 'remaining_payment_months',
 'sum_otherincome',
 'loaninstallment_C',
 'netincome1_business',
 'netincome1_salary',
 'netincome2_business',
 'netincome2_salary',
 'netincome3',
 'affordability1',
 'affordability2',
 'affordability3',
 'housemembers_CC',
 'otherloans_CC',
 'netincome1.2',
 'netincome1.3',
 'Netincome2.3',
 'willing_to_share_phone_number',
 'bad_90']

In [151]:
#list the non-numerical columns
cnames=list(df.select_dtypes(exclude=['number']).columns)
cnames

['farming_duration',
 'animal_numbers',
 'crops_grown',
 'fruit_vegetables_grown',
 'pests_diseases_control',
 'machinery_ownership',
 'farming_acreage',
 'product_purpose',
 'customer_source',
 'family_descision_maker',
 'other_water_source',
 'electricity_connectivity',
 'household_number',
 'occupation_of_head',
 'main_income_source',
 'members_during_peak_season',
 'compensation_type',
 'occupation_length',
 'number_of_businesses_before',
 'duration_with_same_employer',
 'number_of_jobs_before',
 'other_income_source',
 'frequent_monthly_expense',
 'number_of_loans',
 'number_of_outstanding_loans',
 'preferred_banking_method',
 'percentage_who_pay_suppliers_on_time',
 'percentage_who_default',
 'expenditure_after_winning',
 'overall_risk_assessment_by_relationship_manager',
 'sum_householdmember',
 'birth_date',
 'gender',
 'duration_with_phone_number',
 'county_of_residence',
 'duration_in_location',
 'relation_to_next_of_kin',
 'house_hold_number']

In [183]:
#counts
df['bad_90'].value_counts()


0.0    6619
1.0    1162
Name: bad_90, dtype: int64

In [101]:
#data clean up
#Data transformation/cleanup
#Strip off textual parts, represent values as numeric values it makes sense. Convert the datatype to numeric.



In [153]:
#Transform farming duration
df['farming_duration'].replace(to_replace='0-1 year', value='0', regex=True, inplace=True)
df['farming_duration'].replace(to_replace='1-2 years', value='1', regex=True, inplace=True)
df['farming_duration'].replace(to_replace='2-5 years', value='2', regex=True, inplace=True)
df['farming_duration'].replace(to_replace='More than 5 years', value='3', regex=True, inplace=True)
df['farming_duration']= pd.to_numeric(df['farming_duration'], errors='coerce')


C:\Users\Sanjo\anaconda3\envs\myenv371\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\Sanjo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [154]:
#Transform crops_grown
df['crops_grown'].replace(to_replace='0 type of crops', value='0', regex=True, inplace=True)
df['crops_grown'].replace(to_replace='1 type of crop', value='1', regex=True, inplace=True)
df['crops_grown'].replace(to_replace='2 type of crops', value='2', regex=True, inplace=True)
df['crops_grown'].replace(to_replace='3 type of crops', value='3', regex=True, inplace=True)
df['crops_grown'].replace(to_replace='4 type of crops', value='4', regex=True, inplace=True)
df['crops_grown'].replace(to_replace='More than 4 types of animals ', value='5', regex=True, inplace=True)
df['crops_grown'].replace(to_replace='Maize', value='1', regex=True, inplace=True)
df['crops_grown']= pd.to_numeric(df['crops_grown'], errors='coerce')


C:\Users\Sanjo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [155]:
#Transform fruit_vegetables_grown
df['fruit_vegetables_grown'].replace(to_replace='0 type of Fruits', value='0', regex=True, inplace=True)
df['fruit_vegetables_grown'].replace(to_replace='1 type of fruit', value='1', regex=True, inplace=True)
df['fruit_vegetables_grown'].replace(to_replace='2 types of fruits', value='2', regex=True, inplace=True)
df['fruit_vegetables_grown'].replace(to_replace='3 types of fruits', value='3', regex=True, inplace=True)
df['fruit_vegetables_grown'].replace(to_replace='4 types of fruits', value='4', regex=True, inplace=True)
df['fruit_vegetables_grown'].replace(to_replace='More than 4 types of fruits', value='5', regex=True, inplace=True)
df['fruit_vegetables_grown'].replace(to_replace='Other Fruits and Vegetables', value='1', regex=True, inplace=True)
df['fruit_vegetables_grown'].replace(to_replace='Cabbages', value='1', regex=True, inplace=True)
df['fruit_vegetables_grown']= pd.to_numeric(df['fruit_vegetables_grown'], errors='coerce')


C:\Users\Sanjo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [156]:
#Transform animal_numbers
df['animal_numbers'].replace(to_replace='0 type of animals', value='0', regex=True, inplace=True)
df['animal_numbers'].replace(to_replace='1 type of animal', value='1', regex=True, inplace=True)
df['animal_numbers'].replace(to_replace='2 types of animals', value='2', regex=True, inplace=True)
df['animal_numbers'].replace(to_replace='3 types of animals', value='3', regex=True, inplace=True)
df['animal_numbers'].replace(to_replace='4 types of animals', value='4', regex=True, inplace=True)
df['animal_numbers'].replace(to_replace='More than 4 types of crops', value='5', regex=True, inplace=True)
df['animal_numbers'].replace(to_replace='Cows', value='1', regex=True, inplace=True)
df['animal_numbers']= pd.to_numeric(df['animal_numbers'], errors='coerce')


C:\Users\Sanjo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [157]:
#Transform pests_diseases_control
df['pests_diseases_control'].replace(to_replace='No use', value='0', regex=True, inplace=True)
df['pests_diseases_control'].replace(to_replace='Uses every year', value='1', regex=True, inplace=True)
df['pests_diseases_control'].replace(to_replace='Uses in every 2-3 years', value='2', regex=True, inplace=True)
df['pests_diseases_control']= pd.to_numeric(df['pests_diseases_control'], errors='coerce')


C:\Users\Sanjo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [158]:
#Transform machinery_ownership
df['machinery_ownership'].replace(to_replace='Common use without a cost', value='0', regex=True, inplace=True)
df['machinery_ownership'].replace(to_replace='Rented with a cost', value='1', regex=True, inplace=True)
df['machinery_ownership'].replace(to_replace='Owned', value='2', regex=True, inplace=True)
df['machinery_ownership']= pd.to_numeric(df['machinery_ownership'], errors='coerce')


C:\Users\Sanjo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [159]:
#Transform farming_acreage
df['farming_acreage'].replace(to_replace='2', value='0', regex=True, inplace=True)
df['farming_acreage'].replace(to_replace='0.5-3.5 acres', value='0', regex=True, inplace=True)
df['farming_acreage'].replace(to_replace='4-6.5 acres', value='1', regex=True, inplace=True)
df['farming_acreage'].replace(to_replace='More than 7 acres', value='2', regex=True, inplace=True)
df['farming_acreage']= pd.to_numeric(df['farming_acreage'], errors='coerce')


C:\Users\Sanjo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [160]:
#Transform product_purpose
df['product_purpose'].replace(to_replace='Increase of agriculture production for sale', value='0', regex=True, inplace=True)
df['product_purpose'].replace(to_replace='Increase of agriculture production for self consumption', value='1', regex=True, inplace=True)
df['product_purpose'].replace(to_replace='Reduction of household work of fetching water', value='2', regex=True, inplace=True)
df['product_purpose'].replace(to_replace='Acquire a TV for the house', value='3', regex=True, inplace=True)
df['product_purpose'].replace(to_replace='Acquire light for the house', value='4', regex=True, inplace=True)
df['product_purpose']= pd.to_numeric(df['product_purpose'], errors='coerce')


C:\Users\Sanjo\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [161]:
#append 0 to accounts with blank bad_90
New = []
men = df.bad_90.mean()
for val in df.bad_90:
    if ((val == "")):
        New.append(0)
    else:
        New.append(val)

df.bad_90 = New

C:\Users\Sanjo\anaconda3\envs\myenv371\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [168]:
#append 10 to accounts with harvest cycles more than 10
New = []
men = df.harvest_cycles_per_year.mean()
for val in df.harvest_cycles_per_year:
    if ((val >10)):
        New.append(10)
    else:
        New.append(val)

df.harvest_cycles_per_year = New

C:\Users\Sanjo\anaconda3\envs\myenv371\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [171]:
#append 100000 to accounts with salary_amount more than 100,000
New = []
men = df.salary_amount.mean()
for val in df.salary_amount:
    if ((val >100000)):
        New.append(100000)
    else:
        New.append(val)

df.salary_amount = New

In [162]:
#test ml
df_test_data= df[['id_account','willingness_to_record',
 'same_phone_number',
 'land_owner',
 'farming_duration',
 'animal_numbers',
 'harvest_cycles_per_year',
 'crops_grown',
 'fruit_vegetables_grown',
 'pests_diseases_control',
 'machinery_ownership',
 'farming_acreage',
 'product_purpose',
 'hours_fetching_water',
 'pay_for_water',
 'pay_for_pumping',
 'water_needs_capicity',
 'tank_capacity',
 'financial_dependants_children_adults',
 'working_age_adults',
 'pensioners',
 'average_monthly_income',
 'salary_amount',
 'remaining_income_after_expenses',
 'total_outstanding_loan_amount',
 'remaining_payment_months',
 'sum_otherincome',
 'loaninstallment_C',
 'netincome1_business',
 'netincome1_salary',
 'netincome2_business',
 'netincome2_salary',
 'netincome3',
 'affordability1',
 'affordability2',
 'affordability3',
 'housemembers_CC',
 'otherloans_CC',
 'netincome1.2',
 'netincome1.3',
 'Netincome2.3',
 'willing_to_share_phone_number',
 'bad_90']]

In [182]:
df_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8464 entries, 0 to 8463
Data columns (total 43 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id_account                            8464 non-null   int64  
 1   willingness_to_record                 8464 non-null   float64
 2   same_phone_number                     8464 non-null   float64
 3   land_owner                            8464 non-null   float64
 4   farming_duration                      8464 non-null   int64  
 5   animal_numbers                        8322 non-null   float64
 6   harvest_cycles_per_year               8464 non-null   int64  
 7   crops_grown                           8464 non-null   float64
 8   fruit_vegetables_grown                8464 non-null   float64
 9   pests_diseases_control                8464 non-null   int64  
 10  machinery_ownership                   8464 non-null   float64
 11  farming_acreage  

In [175]:
#Missing values imputation

cols = ['willingness_to_record', 'same_phone_number', 'land_owner', 'farming_duration', 'harvest_cycles_per_year',
 'crops_grown', 'fruit_vegetables_grown', 'machinery_ownership','willing_to_share_phone_number']
for col in cols:
    print('Imputation with Median: %s' % (col))    
    df_test_data[col].fillna(df_test_data[col].median(), inplace=True)

cols = ['hours_fetching_water', 'pay_for_water', 'pay_for_pumping', 'water_needs_capicity', 'tank_capacity',
 'financial_dependants_children_adults', 'working_age_adults', 'pensioners', 'average_monthly_income', 'salary_amount',
  'remaining_income_after_expenses']
for col in cols:
    print('Imputation with Mean: %s' % (col))
    df_test_data[col].fillna(df_test_data[col].mean(), inplace=True)
    

cols = ['total_outstanding_loan_amount', 'remaining_payment_months','bad_90']
for col in cols:
    print('Imputation with Zero: %s' % (col))
    df_test_data[col].fillna(0, inplace=True)

print('Missing value imputation done.')

Imputation with Median: willingness_to_record
Imputation with Median: same_phone_number
Imputation with Median: land_owner
Imputation with Median: farming_duration
Imputation with Median: harvest_cycles_per_year
Imputation with Median: crops_grown
Imputation with Median: fruit_vegetables_grown
Imputation with Median: machinery_ownership
Imputation with Median: willing_to_share_phone_number
Imputation with Mean: hours_fetching_water
Imputation with Mean: pay_for_water
Imputation with Mean: pay_for_pumping
Imputation with Mean: water_needs_capicity
Imputation with Mean: tank_capacity
Imputation with Mean: financial_dependants_children_adults
Imputation with Mean: working_age_adults
Imputation with Mean: pensioners
Imputation with Mean: average_monthly_income
Imputation with Mean: salary_amount
Imputation with Mean: remaining_income_after_expenses
Imputation with Zero: total_outstanding_loan_amount
Imputation with Zero: remaining_payment_months
Imputation with Zero: bad_90
Missing value i

In [180]:
df_test_data = df_test_data.apply(pd.to_numeric, errors='coerce')

In [177]:
#Machine Learning
#X and y definitions
X = df_test_data.drop("bad_90", axis=1)
y =  df_test_data.bad_90

In [178]:
# Partition data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"Training data - Features shape: {X_train.shape}")
print(f"Training data - Target shape: {y_train.shape}\n")
print(f"Test data - Features shape: {X_test.shape}")
print(f"Test data - Target shape: {y_test.shape}")

Training data - Features shape: (6771, 42)
Training data - Target shape: (6771,)

Test data - Features shape: (1693, 42)
Test data - Target shape: (1693,)


In [179]:
#We will use the selected features.
#Baseline model
#We will use ROC AUC (AUC from here onwards) to evaluate the model. 
#Let’s get a sense of baseline performance by building a simple model using all the features:

model0 = GradientBoostingClassifier(random_state=42)
model0.fit(X_train, y_train)
def get_roc_auc(model, X, y):
    y_proba = model.predict_proba(X)[:,1]
    return roc_auc_score(y, y_proba)
print(f"Training data - ROC AUC: {get_roc_auc(model0, X_train, y_train):.4f}")
print(f"Test data - ROC AUC: {get_roc_auc(model0, X_test, y_test):.4f}")

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').